In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

In [3]:
#Load the csv exported in Part I to a DataFrame
cities = pd.read_csv("../WeatherPy/weatherdata.csv")
cities.head()

FileNotFoundError: [Errno 2] No such file or directory: '../WeatherPy/weatherdata.csv'

In [5]:

#Configure gmaps.
gmaps.configure(api_key=g_key)

#Humidity Heatmap
humidity = cities["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities[["Lat", "Lng"]]

#Add Heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

In [6]:
#Create new DataFrame fitting weather criteria
#Narrow down the cities to fit weather conditions
#Drop any rows will null values

narrowed_city_df = cities.loc[(cities["Max Temperature"] > 70) & (cities["Max Temperature"] < 80) & (cities["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.reset_index(inplace=True)
del narrowed_city_df['index']
narrowed_city_df.head()

Figure(layout=FigureLayout(height='420px'))

In [7]:
#Store into variable named hotel_df
#Add a "Hotel Name" column to the DataFrame
#Set parameters to search for hotels with 5000 meters
#Hit the Google Places API for each city's coordinates
#Store the first Hotel result into the DataFrame
#Plot markers on top of the heatmap

hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09
81,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09
97,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09
119,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09
134,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09
165,Coahuayana Viejo,18.73,-103.68,71.71,86.0,0.0,3.20,MX,1.605359e+09
194,Jalu,29.03,21.55,77.36,36.0,0.0,8.97,LY,1.605359e+09
208,Nālūt,30.33,10.85,72.12,27.0,0.0,1.68,LY,1.605359e+09
221,Jaguaruna,-28.61,-49.03,79.25,63.0,0.0,4.68,BR,1.605359e+09
256,Mackay,-21.15,149.20,77.00,88.0,0.0,6.93,AU,1.605359e+09


In [19]:
# NOTE: Do not change any code in this cell
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09
81,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09
97,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09
119,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09
134,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09


In [23]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

City          20
Lat           20
Lng           20
Max Temp      20
Humidity      20
Cloudiness    20
Wind Speed    20
Country       20
Date          20
dtype: int64

In [28]:
#Add a "Hotel Name" column to the DataFrame.
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.
#Store the first Hotel result into the DataFrame.
#Plot markers on top of the heatmap.


hotel_df['Hotel Name'] = ''
locations = hotel_df[['Lat', 'Lng']]
hotel_df = hotel_df.reset_index(drop=True)
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09,
1,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09,
2,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09,
3,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09,
4,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09,


In [25]:

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {
    'rankby': 'distance',
    'type': 'lodging',
    'key': g_key,
    'keyword': 'hotel'
}

for index, row in hotel_df.iterrows():
    location = f'{row["Lat"]},{row["Lng"]}'
    params['location'] = location
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except(SyntaxError, IndexError):
        print('sorry, no luck')

sorry, no luck
sorry, no luck


In [31]:
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Awjilah,29.11,21.29,77.07,36.0,0.0,7.92,LY,1.605359e+09,
1,Sabha,27.04,14.43,74.05,31.0,0.0,7.85,LY,1.605359e+09,
2,Broken Hill,-31.95,141.43,75.20,22.0,0.0,5.82,AU,1.605359e+09,
3,Riyadh,24.69,46.72,77.00,25.0,0.0,4.70,SA,1.605359e+09,
4,Hervey Bay,-25.30,152.85,77.00,83.0,0.0,5.99,AU,1.605359e+09,
